In [ ]:
import pandas as pd
import glob
import math
import os
os.chdir('/Users/denislukanov/Desktop/Causal/data')
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('seaborn-deep')
import numpy as np
from matplotlib.pyplot import figure
import datetime
import math
import sklearn
import pandas_datareader.data as web
import sklearn.discriminant_analysis
#from pandas.io.data import DataReader
from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from sklearn.metrics import confusion_matrix
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.svm import LinearSVC, SVC
# from pandas_datareader import data as pdr
# import yfinance as yfin
import fattails.metrics as fattails
import lightgbm as lgb
#yfin.pdr_override()
#import matplotlib.pyplot as plt
import seaborn as sns
import ta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
import shap

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
data1 = pd.read_csv('BTCUSDT_15M.csv', index_col=0)
data1=data1[['open', 'high', 'low', 'close', 'vol']]

In [ ]:
data1

In [ ]:
ts=data1

ts['pct_change'] = ts.close.pct_change()
ts['survival_probability'] = fattails.get_survival_probability(ts['pct_change'])
ts['dollar_value']=ts['close']*ts['vol']
ts['dollar_value_ch']=ts['dollar_value']-ts['dollar_value'].shift(1)
ts['log_ret']=np.log(ts['close']).diff()
ts=ts.dropna()


In [ ]:
def get_gap_OC(df_open,df_close):    
    dif = []
    for i in range(len(df_open)):
        a=df_open[i]-df_close[i-1]
        dif.append(a)
    return dif

In [ ]:
def R_method(high, low, close, window_size):
    
    # Calculate the rolling average of high and low prices for the last window_size days
    high_rolling_mean = high.rolling(window=window_size).mean()
    low_rolling_mean = low.rolling(window=window_size).mean()
    
    # Convert the close column to float64 data type
    close = close.astype(float)
    
    # Calculate the R method value
    R_method = (high_rolling_mean - close) / (high_rolling_mean - low_rolling_mean)
    
    # Return the R method value
    return R_method
R = R_method(high=data1['high'], low=data1['low'], close=data1['close'], window_size=20)

In [ ]:
def historicalVaR(returns, alpha=5):
    """
    Read in a pandas dataframe of returns / a pandas series of returns.
    Output the percentile of the distribution at the given alpha confidence level.
    """
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    
    # A passed user-defined-function will be passed a Series for evaluation.
    
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalVaR, alpha=alpha)
   
    else:
        raise TypeError("Expected returns to be dataframe or series")

In [ ]:
def historicalCVaR(returns, alpha=5):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the CVaR for dataframe / series
    """
    if isinstance(returns, pd.Series):
        belowVaR = returns <= historicalVaR(returns, alpha=alpha)
        return returns[belowVaR].mean()
    
    # A passed user-defined-function will be passed a Series for evaluation.
    
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalCVaR, alpha=alpha)
   
    else:
        raise TypeError("Expected returns to be dataframe or series")

In [ ]:
def getDailyVol(close,span0=100):
    close.index = pd.to_datetime(close.index)
    
    df0=close.index.searchsorted(close.index-pd.Timedelta(days=3))
    df0=df0[df0>0]
    df0=pd.Series(close.index[df0-1],index=close.index[close.shape[0]-df0.shape[0]:])
    df0=close.loc[df0.index]/close.loc[df0.values].values-1#daily returns
    df0=df0.ewm(span=span0).std()
    #df0=df0.rename(columns={'Close':'volatilities'})
    return df0

In [ ]:
gap_OC=get_gap_OC(ts['open'],ts['close'])
ts['gap_OC']=gap_OC
ts['gap_OC'][0]=0
ts['H-L_spread_change']=(ts['high']-ts['low'])/ts['close']
ts['O-C_spread_change']=(ts['open']-ts['close'])/ts['close']
ts['R_indicator'] = R

ts['rolling_dollar_volume']=ts['dollar_value'].rolling(16).mean() 
ts['CVar_16']=ts['pct_change'].rolling(16).apply(historicalCVaR)
ts['CVar_4']=ts['pct_change'].rolling(4).apply(historicalCVaR)
ts["Direction"] = np.sign(ts["pct_change"])
ts["Direction_1Lag"] = np.sign(ts["pct_change"].shift(+1))
ts["Volatility"] = getDailyVol(ts['close'],span0=10)

ts['rolling_max']=ts['close'].rolling(7).max()
ts['rolling_min']=ts['close'].rolling(7).min()
ts['MA_4']=ts['close'].rolling(4).mean()
ts['MA_16']=ts['close'].rolling(16).mean()

ts['Med_16']=ts['close'].rolling(16).median()
ts['sd_4']=ts['close'].rolling(4).std()
ts['sd_16']=ts['close'].rolling(16).std()
ts['sk_4']=ts['close'].rolling(4).skew()
ts['sk_16']=ts['close'].rolling(16).skew()

In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
ts=clean_dataset(ts)

In [ ]:
#Calculating "from package" tech indicators

In [ ]:

from ta.trend import MACD
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange
from ta.momentum import StochasticOscillator
from ta.trend import ADXIndicator
from ta.trend import IchimokuIndicator
from ta.volume import VolumeWeightedAveragePrice


vwap = VolumeWeightedAveragePrice(ts['close'], ts['high'], ts['low'], ts['vol'])
ts['vwap'] = vwap.volume_weighted_average_price()

ichimoku = IchimokuIndicator(ts['high'], ts['low'])
ts['tenkan_sen'] = ichimoku.ichimoku_conversion_line()
ts['kijun_sen'] = ichimoku.ichimoku_base_line()
ts['senkou_span_a'] = ichimoku.ichimoku_a()
ts['senkou_span_b'] = ichimoku.ichimoku_b()


adx = ADXIndicator(ts['high'], ts['low'], ts['close'])
ts['adx'] = adx.adx()

stoch = StochasticOscillator(ts['high'], ts['low'], ts['close'])
ts['stoch'] = stoch.stoch()

# atr = AverageTrueRange(ts['high'], ts['low'], ts['close'])
# ts['atr'] = atr.average_true_range()

bb = BollingerBands(ts['close'])
ts['bb_upperband'] = bb.bollinger_hband()
ts['bb_lowerband'] = bb.bollinger_lband()

rsi = RSIIndicator(ts['close'])
ts['rsi'] = rsi.rsi()

macd = MACD(ts['close'])
ts['macd'] = macd.macd()
ts['macd_signal'] = macd.macd_signal()
ts['macd_diff'] = macd.macd_diff()

ts['atr']=ta.volatility.average_true_range(ts['high'], ts['low'], ts['close'])


In [ ]:
ts.shape

In [ ]:
ts=clean_dataset(ts)

In [ ]:
#Filtering out by volume doesn't affect a lot the performance
#ts=ts[ts['dollar_value']>ts['rolling_dollar_volume']]

In [ ]:
# Searching for a quasi-treatment via different regimes by separting on quantile bins on different features

In [ ]:
ts['pct_change_bins'], pct_change_edges = pd.qcut(ts['pct_change'], q=10, labels=False, retbins=True)
ts['CVar_4_bins'], CVar_4_bin_edges = pd.qcut(ts['CVar_4'], q=10, labels=False, retbins=True)
ts['CVar_16_bins'], CVar_16_bin_edges = pd.qcut(ts['CVar_16'], q=10, labels=False, retbins=True)
ts['dollar_value_ch_bins'], dollar_value_ch_bin_edges = pd.qcut(ts['dollar_value_ch'], q=10, labels=False, retbins=True)
ts['Volatility_bins'], volatility_bin_edges = pd.qcut(ts['Volatility'], q=10, labels=False, retbins=True)
ts['Skeness_bins'], skew_bin_edges = pd.qcut(ts['sk_4'], q=10, labels=False, retbins=True)
ts['Macd_bins'], macd_bin_edges = pd.qcut(ts['macd'], q=10, labels=False, retbins=True)
ts['atr_bins'], atr_bin_edges = pd.qcut(ts['atr'], q=10, labels=False, retbins=True)

print("Pcnt_change Bin Ranges:")
for i in range(len(pct_change_edges) - 1):
    print(f"Bin {i}: {pct_change_edges[i]} - {pct_change_edges[i+1]}")

print("\nCVar_4 Bin Ranges:")
for i in range(len(CVar_4_bin_edges) - 1):
    print(f"Bin {i}: {CVar_4_bin_edges[i]} - {CVar_4_bin_edges[i+1]}")
    
print("\nCVar_16 Bin Ranges:")
for i in range(len(CVar_16_bin_edges) - 1):
    print(f"Bin {i}: {CVar_16_bin_edges[i]} - {CVar_16_bin_edges[i+1]}")
      
print("\nDollar_value_ch_bin_edges  Bin Ranges:")
for i in range(len(dollar_value_ch_bin_edges) - 1):
    print(f"Bin {i}: {dollar_value_ch_bin_edges [i]} - {dollar_value_ch_bin_edges [i+1]}") 

print("\nVolatility  Bin Ranges:")
for i in range(len(volatility_bin_edges) - 1):
    print(f"Bin {i}: {volatility_bin_edges [i]} - {volatility_bin_edges [i+1]}") 
    
print("\nSkeweness  Bin Ranges:")
for i in range(len(volatility_bin_edges) - 1):
    print(f"Bin {i}: {skew_bin_edges [i]} - {skew_bin_edges [i+1]}") 

print("\nMacd  Bin Ranges:")
for i in range(len(volatility_bin_edges) - 1):
    print(f"Bin {i}: {macd_bin_edges [i]} - {macd_bin_edges [i+1]}") 

print("\nATR  Bin Ranges:")
for i in range(len(volatility_bin_edges) - 1):
    print(f"Bin {i}: {atr_bin_edges [i]} - {atr_bin_edges [i+1]}") 
   
   
    

In [ ]:
# Binning the data
ts['pct_change_bins'], pct_change_edges = pd.qcut(ts['pct_change'], q=10, labels=False, retbins=True)
ts['CVar_4_bins'], CVar_4_bin_edges = pd.qcut(ts['CVar_4'], q=10, labels=False, retbins=True)
ts['CVar_16_bins'], CVar_16_bin_edges = pd.qcut(ts['CVar_16'], q=10, labels=False, retbins=True)
ts['dollar_value_ch_bins'], dollar_value_ch_bin_edges = pd.qcut(ts['dollar_value_ch'], q=10, labels=False, retbins=True)
ts['Volatility_bins'], volatility_bin_edges = pd.qcut(ts['Volatility'], q=10, labels=False, retbins=True)
ts['Skeness_bins'], skew_bin_edges = pd.qcut(ts['sk_4'], q=10, labels=False, retbins=True)
ts['Macd_bins'], macd_bin_edges = pd.qcut(ts['macd'], q=10, labels=False, retbins=True)
ts['atr_bins'], atr_bin_edges = pd.qcut(ts['atr'], q=10, labels=False, retbins=True)

# Displaying the bin ranges in a table
bin_ranges = {
    'Pcnt_change': pct_change_edges,
    'CVar_4': CVar_4_bin_edges,
    'CVar_16': CVar_16_bin_edges,
    'Dollar_value_ch': dollar_value_ch_bin_edges,
    'Volatility': volatility_bin_edges,
    'Skewness': skew_bin_edges,
    'Macd': macd_bin_edges,
    'ATR': atr_bin_edges
}

bin_ranges_df = pd.DataFrame(bin_ranges)
bin_ranges_df.index.name = 'Bin'
bin_ranges_df.reset_index(inplace=True)

bin_ranges_df


In [ ]:
# Further I choose best bins given there separation through visualisation

In [ ]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='CVar_4_bins', y='pct_change', data=ts);


In [ ]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='CVar_16_bins', y='pct_change', data=ts);


In [ ]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='CVar_16_bins', y='pct_change', data=ts);

In [ ]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='Volatility_bins', y='dollar_value_ch', data=ts);

In [ ]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='Skeness_bins', y='pct_change', data=ts);

In [ ]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='Macd_bins', y='pct_change', data=ts);

In [ ]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='atr_bins', y='pct_change', data=ts);

In [ ]:
# Next I make default correlation on uncleaned potential features space to search for singular ones

In [ ]:
corr_matrix =ts.corr()

plt.figure(figsize=(30,20))  
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True)

# Add title and adjust the layout
plt.title('Correlation Heatmap')
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
ts.columns

In [ ]:
ts_1=ts[[ 'vol', 'survival_probability', 'dollar_value', 'dollar_value_ch', 'log_ret',
       'gap_OC', 'H-L_spread_change', 'O-C_spread_change', 'R_indicator',
       'rolling_dollar_volume', 'CVar_16', 'CVar_4', 'Direction',
       'Direction_1Lag', 'Volatility', 'sd_4', 'sd_16', 'sk_4', 'sk_16', 'vwap',
        'adx','stoch', 'rsi', 'macd','atr']]

In [ ]:
corr_matrix =ts_1.corr()

plt.figure(figsize=(30,20))  
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True)

# Add title and adjust the layout
plt.title('Correlation Heatmap')
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
ts_1.columns

In [ ]:
colums_to_scale=['vol', 'survival_probability', 'dollar_value', 'dollar_value_ch',
       'log_ret', 'gap_OC', 'H-L_spread_change', 'O-C_spread_change',
       'R_indicator', 'rolling_dollar_volume', 'CVar_16', 'CVar_4',
       'Direction', 'Direction_1Lag', 'Volatility', 'sd_4', 'sd_16', 'sk_4',
       'sk_16', 'vwap', 'adx', 'stoch', 'rsi', 'macd','atr']

In [ ]:
# Initialize the MinMaxScaler that is better for TS
scaler = MinMaxScaler()

# Fit and transform the DataFrame
ts_1 = pd.DataFrame(scaler.fit_transform(ts_1[colums_to_scale]), columns=colums_to_scale)

In [ ]:
ts_1['CVar_4_bins']=ts['CVar_4_bins'].values

In [ ]:
# The aim of algpo below is to detect non-linear correlation via the powers of exponent
# Ultimately it mainly repeats what default linear corr does

In [ ]:
# %%time

# columns_list = []

# for i, col1 in enumerate(ts_1.columns):
#     for j, col2 in enumerate(ts_1.columns):
#         if i != j:  # Exclude diagonal elements
#             for t in range(1, 10):
#                 for s in range(1, 10):
#                     t0 = t / 10
#                     s0 = s / 10
#                     n = 1000
#                     A = 0
#                     B = 0
#                     for k in range(n):
#                         A += math.exp(t0 * ts_1[col1][k])
#                         B += math.exp(s0 * ts_1[col2][k])
#                     A = A / n
#                     B = B / n
#                     covnl = 0
#                     s1 = 0
#                     s2 = 0
#                     for k in range(n):
#                         covnl += (math.exp(t0 * ts_1[col1][k]) - A) * (math.exp(s0 * ts_1[col2][k]) - B)
#                         s1 += (math.exp(t0 * ts_1[col1][k]) - A) ** 2
#                         s2 += (math.exp(s0 * ts_1[col2][k]) - B) ** 2
#                     covnl = covnl / n
#                     s1 = math.sqrt(s1 / n)
#                     s2 = math.sqrt(s2 / n)
#                     if covnl / (s1 * s2) > 0.4 or covnl / (s1 * s2) < -0.4:
#                         print(t0, s0, col1, col2, covnl / (s1 * s2))
#                         columns_list.append((col1, col2))

# print("Columns that meet the criteria:")
# for columns in columns_list:
#     print(columns)


In [ ]:
# unique_list = list(set(columns_list))
# unique_names = set()
# for pair in unique_list:
#     unique_names.add(pair[0])
#     unique_names.add(pair[1])

# unique_names_list = list(unique_names)

In [ ]:
# unique_names_list

In [ ]:
ts_1= clean_dataset(ts_1)

In [ ]:
ts_1['CVar_4_bins'].value_counts()

In [ ]:
ts_2=ts_1[ts_1['CVar_4_bins']==0]

In [ ]:
ts_2.columns

In [ ]:
features_space = list(['vol', 'survival_probability', 'dollar_value',
       'dollar_value_ch', 'log_ret', 'gap_OC', 'H-L_spread_change',
       'O-C_spread_change', 'R_indicator', 'rolling_dollar_volume', 'CVar_16',
       'CVar_4', 'Direction', 'Direction_1Lag', 'Volatility', 'sd_4', 'sd_16',
       'sk_4', 'sk_16', 'vwap', 'adx', 'stoch', 'rsi', 'macd',
        'atr'])
features_space

In [ ]:
def acper(y_true, y_pred):
    """
    This function calculates Almost Correct Predictions Error Rate (ACPER)
    :param y_true: list of real numbers, true values
    :param y_pred: list of real numbers, predicted values
    :returns: acper score
    """
    threshold = 0.05
    for yt, yp in zip(y_true, y_pred):
        lower_bound = yt - (threshold * yt)
        upper_bound = yt + (threshold * yt)
        if (yp <= upper_bound) & (yp >= lower_bound): 
            yield True
        else:
            yield False

In [ ]:
def model_dev(dfs, feats_space, target):
    
    n_splits=3
    kf = KFold(n_splits)
    
    for fold, (train_index, test_index) in enumerate(kf.split(dfs)):
        print (f"\n fold {fold}")
        print(f" Target is: {target} ")
        print()
        

        X_train, X_test = dfs[feats_space].iloc[train_index[:-1]], dfs[feats_space].iloc[test_index[:-1]]
        y_train, y_test = dfs[target].iloc[train_index[1:]], dfs[target].iloc[test_index[1:]]
        train_dataset = lgb.Dataset(X_train, y_train)
        val_dataset = lgb.Dataset(X_test, y_test)

        params = {'objective': 'mae', 'boosting_type': 'gbdt', 'max_depth': -1,'feature_fraction': .8, 
        'feature_fraction_bynode': .8, 'bagging_fraction': 0.8, 'bagging_freq': 42, 'n_jobs': -1, 'verbose': -1}

        model = lgb.train(params = params, train_set = train_dataset, 
                          valid_sets = val_dataset, 
                          num_boost_round = 1000, 
                          early_stopping_rounds = 100, 
                          verbose_eval = 100)

        preds=model.predict(X_test)
        score1 = mean_absolute_error(y_test, preds)
        shap_values = shap.TreeExplainer(model).shap_values(X_test)
        shap.summary_plot(shap_values, X_test)
        print(np.abs(shap_values).mean(0)[0])
        # Visualize the LightGBM tree model
        lgb.plot_tree(model, figsize=(20, 10), show_info=['internal_count'],precision=3, orientation='horizontal')
        plt.show()
        
        vals= np.abs(shap_values).mean(0)
        f_imp = pd.DataFrame(list(zip(X_test.columns,vals)),columns=['col_name',f"feature_importance_vals{fold}"])
        
        f_imp.sort_values(by=[f"feature_importance_vals{fold}"],ascending=False,inplace=True)
        f_imp.reset_index(drop = True, inplace = True)
        
        if fold == 0:
            features_importance = f_imp.copy()
        else:
            features_importance = pd.merge(features_importance, f_imp, how = 'left', on = 'col_name')
        
        print(f_imp.head())
        #scores1 += score1
        #means =X_train.mean()
        print(score1)
        
        y_true = y_test
        y_pred = preds

        T_F=list(acper(y_true, y_pred))
        true = T_F.count(True)
        ratio=true/len(T_F)
        print('The ratio of almost correct to incorrect is ', ratio)
        
    imp_cols = [c for c in features_importance.columns if c != "col_name"]
    for col in imp_cols:
        features_importance[col] = np.abs(features_importance[col])
    
    #ratio=true/len(T_F)
    features_importance['mean'] = features_importance[imp_cols].mean(axis = 1)
    features_importance.sort_values('mean', ascending = False, inplace = True)
    # to select least imp feature make ascending = True
    features_importance.reset_index(drop = True, inplace = True)
    return [features_importance['col_name'][0],features_importance['mean'][0]] # returns most important features by means across folds by shap values with a mean value
    #return features_importance['col_name'][0],features_importance['col_name'][1] # returns 2 most imp feats
    #return features_importance #returns all features by iteration with values
    #return features_importance['mean'][0]
    tree_df = model.booster_.trees_to_dataframe()

In [ ]:
# Going through the whole feat space with removing target from feature space

In [ ]:
%%time

l=[]
targ=[]

for f in features_space:
    feats_space = [feat for feat in features_space if feat != f]
    target = f
    targ.append(f)
    
    l += [model_dev(ts_2, feats_space, target)]

In [ ]:
#Removing 1st most important feature

In [ ]:
%%time

l_1=[]

for i in range(len(features_space)):
    f_0 = features_space[i]
    f_1=l[i]
    
    feats_space = [feat for feat in features_space if feat not in f_0 and feat not in f_1]
    target = f_0
    l_1 += [model_dev(ts_2, feats_space, target)]


In [ ]:
#first and second most important features are removed

In [ ]:
%%time

l_2=[]
targ=[]

for i in range(len(features_space)):
    f_0 = features_space[i]
    f_1=l[i]
    f_2=l_1[i]
    
    feats_space = [feat for feat in features_space if feat not in f_0 and feat not in f_1 and feat not in f_2]
    target = f_0
    l_2 += [model_dev(ts_2, feats_space, target)]

In [ ]:
# 1st, 2nd an 3d most important features are removed

In [ ]:
%%time

l_3=[]
targ=[]

for i in range(len(features_space)):
    f_0 = features_space[i]
    f_1=l[i]
    f_2=l_1[i]
    f_3=l_2[i]
    
    feats_space = [feat for feat in features_space if feat not in f_0 and feat not in f_1 and feat not in f_2 and feat not in f_3]
    target = f_0
    l_3 += [model_dev(ts_2, feats_space, target)]

In [ ]:
# 1-4 most important features are removed

In [ ]:
%%time

l_4=[]
targ=[]

for i in range(len(features_space)):
    f_0 = features_space[i]
    f_1=l[i]
    f_2=l_1[i]
    f_3=l_2[i]
    f_4=l_3[i]
    
    feats_space = [feat for feat in features_space if feat not in f_0 and feat not in f_1 \
                   and feat not in f_2 and feat not in f_3 and feat not in f_4]
    target = f_0
    l_4 += [model_dev(ts_2, feats_space, target)]

In [ ]:
feat_targ=pd.DataFrame(data=[targ,l,l_1,l_2,l_3,l_4]).T
feat_targ.columns =['Target', '1st imp feat', '2nd imp feat','3rd imp feat','4th imp feat',\
                   '5th imp feat']
feat_targ['Target'] = features_space
feat_targ

In [ ]:
#Next the same type of procedure is done for regime on "another end"

In [ ]:
ts_3=ts_1[ts_1['CVar_4_bins']==9]

In [ ]:
features_space = list(['vol', 'survival_probability', 'dollar_value',
       'dollar_value_ch', 'log_ret', 'gap_OC', 'H-L_spread_change',
       'O-C_spread_change', 'R_indicator', 'rolling_dollar_volume', 'CVar_16',
       'CVar_4', 'Direction', 'Direction_1Lag', 'Volatility', 'sd_4', 'sd_16',
       'sk_4', 'sk_16', 'vwap', 'adx', 'stoch', 'rsi', 'macd',
        'atr'])
features_space

In [ ]:
ts_3

In [ ]:
%%time

l=[]
targ=[]

for f in features_space:
    feats_space = [feat for feat in features_space if feat != f]
    target = f
    targ.append(f)
    
    l += [model_dev(ts_3, feats_space, target)]

In [ ]:
%%time

l_1=[]

for i in range(len(features_space)):
    f_0 = features_space[i]
    f_1=l[i]
    
    feats_space = [feat for feat in features_space if feat not in f_0 and feat not in f_1]
    target = f_0
    l_1 += [model_dev(ts_3, feats_space, target)]


In [ ]:
%%time

l_2=[]
targ=[]

for i in range(len(features_space)):
    f_0 = features_space[i]
    f_1=l[i]
    f_2=l_1[i]
    
    feats_space = [feat for feat in features_space if feat not in f_0 and feat not in f_1 and feat not in f_2]
    target = f_0
    l_2 += [model_dev(ts_2, feats_space, target)]

In [ ]:
%%time

l_3=[]
targ=[]

for i in range(len(features_space)):
    f_0 = features_space[i]
    f_1=l[i]
    f_2=l_1[i]
    f_3=l_2[i]
    
    feats_space = [feat for feat in features_space if feat not in f_0 and feat not in f_1 and feat not in f_2 and feat not in f_3]
    target = f_0
    l_3 += [model_dev(ts_2, feats_space, target)]

In [ ]:
%%time

l_4=[]
targ=[]

for i in range(len(features_space)):
    f_0 = features_space[i]
    f_1=l[i]
    f_2=l_1[i]
    f_3=l_2[i]
    f_4=l_3[i]
    
    feats_space = [feat for feat in features_space if feat not in f_0 and feat not in f_1 \
                   and feat not in f_2 and feat not in f_3 and feat not in f_4]
    target = f_0
    l_4 += [model_dev(ts_2, feats_space, target)]

In [ ]:
feat_targ=pd.DataFrame(data=[targ,l,l_1,l_2,l_3,l_4]).T
feat_targ.columns =['Target', '1st imp feat', '2nd imp feat','3rd imp feat','4th imp feat',\
                   '5th imp feat']
feat_targ['Target'] = features_space
feat_targ